In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.patches
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import pandas as pd
import flopy

In [ ]:
# define model parameters

exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = Path(f"./mf6_pinder_sauer")

leakage = True

Lx = 427
Ly = 39624
dx = 7 * [28.3] + [30.48] + 7 * [28.3]
dy = 609.61
nrow = 65
ncol = 15
nlay = 1

# initial water table
head_upstream = 67.05
head_downstream = 27.43
water_table_slope = (head_downstream - head_upstream) / Ly
y = np.linspace(.5 * dy, nrow * dy - 0.5 * dy, nrow)
strt = head_upstream + y * water_table_slope
strt = np.repeat(strt, ncol).reshape(nlay, nrow, ncol)

iobs_row = 24 # zero-based row number for comparing stream outflow
iobs_col = 7 # column with channel in it

sy = 0.25
ss = 0.25
kx = 3.048e-3

leakage_coefficient = 1.402e-4

channel_bedslope = 0.001
channel_roughness = 0.03858 # m-s units
Q0 = 509.7 # m^3/s
depth0 = 6.09
zdg_slope = 0.001

gwf_botm = 0

nper = 1
perlen = 24 * 60 * 60 # simulation period is 24 hours
nstp = int(perlen / (5 * 60)) # time steps are 5 minutes
perlen = [perlen]
nstp = nper * [nstp]
tsmult = nper * [1.0]

tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

name = "swf"

idomain = np.zeros((nrow, ncol), dtype=int)
idomain[:, iobs_col] = 1

In [ ]:
def qinflow_func(t):
    Tp = 18000 # seconds
    return 509.7 + 141.58 * np.sin(2 * np.pi * t / Tp)
qinflow_times = np.arange(0, 24 * 60 * 60 + 5 * 60)
qinflow = qinflow_func(qinflow_times)
plt.plot(qinflow_times / 60, qinflow)
plt.xlabel("time, in hours")
plt.ylabel("inflow, in cubic meters per second")

In [ ]:
def add_swf_model(sim):
    # surface water model
    swfname = f"swf_model"
    swf = flopy.mf6.ModflowSwf(sim, modelname=swfname, save_flows=True)

    dis = flopy.mf6.ModflowSwfdis2D(
        swf,
        nrow=nrow,
        ncol=ncol,
        delr=dx,
        delc=dy,
        botm=strt,
        idomain=idomain
    )

    dfw = flopy.mf6.ModflowSwfdfw(
        swf,
        print_flows=False,
        save_flows=True,
        save_velocity=True,
        manningsn=channel_roughness,
        idcxs=None,
    )

    sto = flopy.mf6.ModflowSwfsto(
        swf,
        save_flows=True,
        steady_state={0: False, 1: False},
        transient={0: True, 1: True},
    )

    ic = flopy.mf6.ModflowSwfic(
        swf,
        strt=strt + depth0,
    )

    # output control
    oc = flopy.mf6.ModflowSwfoc(
        swf,
        budget_filerecord=f"{swfname}.bud",
        stage_filerecord=f"{swfname}.stage",
        saverecord=[
            ("STAGE", "ALL"),
            ("BUDGET", "ALL"),
        ],
        printrecord=[
            ("BUDGET", "ALL"),
        ],
    )

    # flw
    spd = [(0, int(ncol / 2), "qinflow")]
    flw = flopy.mf6.ModflowSwfflw(
        swf,
        maxbound=len(spd),
        print_input=True,
        print_flows=True,
        stress_period_data={0: spd},
    )
    fname = f"{swfname}.flw.ts"
    flw.ts.initialize(
        filename=fname,
        timeseries=list(zip(qinflow_times, qinflow)),
        time_series_namerecord=["qinflow"],
        interpolation_methodrecord=["linearend"],
    )

    fname = f"{swfname}.outflow.csv"
    zdg_obs = {
        fname: [
            ("OUTFLOW", "ZDG", (nrow - 1, int(ncol / 2))),
        ],
        "digits": 10,
    }

    idcxs = -1  # use cross section 0
    width = dx[7]
    spd = [
        ((nrow - 1, int(ncol / 2)), idcxs, width, zdg_slope, channel_roughness)
    ]
    zdg = flopy.mf6.ModflowSwfzdg(
        swf,
        observations=zdg_obs,
        print_input=True,
        maxbound=len(spd),
        stress_period_data=spd,
    )

    obs_data = {
        f"{swfname}.obs.csv": [
            ("STAGE", "STAGE", (iobs_row, iobs_col)),
            ("STAGE0", "STAGE", (0, iobs_col)),
            ("FLOW", "FLOW-JA-FACE", (iobs_row, 7), (iobs_row + 1, iobs_col)),
            ("FLOW0", "FLOW-JA-FACE", (0, iobs_col), (1, iobs_col)),
        ],
    }
    obs_package = flopy.mf6.ModflowUtlobs(
        swf,
        filename=f"{swfname}.obs",
        digits=10,
        print_input=True,
        continuous=obs_data,
    )

    return swf

In [ ]:
def add_gwf_model(sim):

    # create gwf model
    name = "gwf_model"
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=name,
        save_flows=True,
        newtonoptions="UNDER_RELAXATION",
    )

    dis = flopy.mf6.ModflowGwfdis(
        gwf,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=dx,
        delc=dy,
        top=strt,
        botm=gwf_botm,
    )

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(
        gwf, 
        strt=strt + depth0
    )

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_flows=False,
        icelltype=0,
        k=kx,
    )

    sto = flopy.mf6.ModflowGwfsto(
        gwf, 
        sy=sy, 
        ss=ss, 
        iconvert=0,
        steady_state={0: False, 1: False},
        transient={0: True, 1: True},        
    )

    # output control
    oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=f"{name}.cbc",
        head_filerecord=f"{name}.hds",
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )
    return gwf


In [ ]:
def add_exchange(sim):
    swfgwf_data = []
    j = iobs_col
    for i in range(nrow):
        cfact = dx[j] * dy
        exg = ((i, j), (0, i, j), leakage_coefficient, cfact)
        swfgwf_data.append(exg)
    swfgwf = flopy.mf6.ModflowSwfgwf(
        sim,
        print_input=False,
        print_flows=True,
        exgtype="SWF6-GWF6",
        nexg=len(swfgwf_data),
        exgmnamea="swf_model",
        exgmnameb="gwf_model",
        exchangedata=swfgwf_data,
    )
    return swfgwf

In [ ]:
# make modflow model
# build MODFLOW 6 files
name = "mysim"
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    version="mf6",
    exe_name=exe_name,
    sim_ws=sim_ws,
)

# create tdis package
tdis = flopy.mf6.ModflowTdis(
    sim,
    ats_filerecord=ats_filerecord,
    time_units="SECONDS",
    nper=nper,
    perioddata=tdis_rc,
)

# add swf model
swf = add_swf_model(sim)

# add gwf model
if leakage:
    gwf = add_gwf_model(sim)
    exg = add_exchange(sim)
else:
    gwf = None
    exg = None

nouter, ninner = 100, 15
hclose, relax = 1e-6, 0.97
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="DBD",
    under_relaxation_theta=0.9,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    backtracking_number=5,
    backtracking_tolerance=1.0,
    backtracking_reduction_factor=0.3,
    backtracking_residual_limit=100.0,
    filename=f"{name}.ims",
)
model_list = [swf.name]
if gwf is not None:
    model_list.append(gwf.name)
sim.register_ims_package(ims, model_list)

# write the model files
sim.write_simulation()

# run the model
sim.run_simulation()

In [ ]:
fpth = sim_ws / f"swf_model.outflow.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    obsvals["time"] / 60.0,
    -obsvals["OUTFLOW"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"MODFLOW 6",
)
#ax.plot([90, 90], [0, 5], "k--")
#ax.set_xlim(0, 180.0)
#ax.set_ylim(0, 5)
#ax.set_aspect(20)
plt.xlabel("Time, in minutes")
plt.ylabel("Discharge,\nin cubic meters per second")
plt.legend(loc="upper right", fontsize=7)
plt.title("Outflow at ZDG Boundary")
fname = Path("../../doc/figures/") / "vcatchgwf-results.png"
# plt.savefig(fname)

In [ ]:
fpth = sim_ws / f"swf_model.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    obsvals["time"] / 60.0,
    -obsvals["FLOW"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"row 1",
)
ax.plot(
    obsvals["time"] / 60.0,
    -obsvals["FLOW0"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"row {iobs_row + 1}",
)
#ax.plot([90, 90], [0, 5], "k--")
#ax.set_xlim(0, 180.0)
#ax.set_ylim(0, 5)
#ax.set_aspect(20)
plt.xlabel("Time, in minutes")
plt.ylabel("Discharge,\nin cubic meters per second")
plt.legend(loc="upper right", fontsize=7)
fname = Path("../../doc/figures/") / "vcatchgwf-results.png"
# plt.savefig(fname)

In [ ]:
fpth = sim_ws / f"swf_model.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    obsvals["time"] / 60.0,
    obsvals["STAGE"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"row {iobs_row + 1}",
)
ax.plot(
    obsvals["time"] / 60.0,
    obsvals["STAGE0"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"row {0 + 1}",
)
#ax.plot([90, 90], [0, 5], "k--")
#ax.set_xlim(0, 180.0)
#ax.set_ylim(0, 5)
#ax.set_aspect(20)
plt.xlabel("Time, in minutes")
plt.ylabel("Stage, in meters")
plt.legend(loc="upper right", fontsize=7)
fname = Path("../../doc/figures/") / "vcatchgwf-results.png"
# plt.savefig(fname)

In [ ]:
def qanal_func(t, leakage=False):
    l1 = -4.799e-2
    l2 = -0.3608
    if leakage:
        l1 = -0.1785
        l2 = -0.3409
    x = 15240.
    d = 4894.3
    fr = 3.49e-4
    return 509.70 + 141.58 * np.exp(l1 * x / d) * np.sin(fr * t + l2 * x / d)

In [ ]:
fpth = sim_ws / f"swf_model.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    obsvals["time"] / 60.0 / 60,
    -obsvals["FLOW"] - Q0,
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"MODFLOW 6 - {"" if leakage else "no "}leakage",
)
ax.plot(
    obsvals["time"] / 60.0 / 60,
    qanal_func(obsvals["time"], leakage=leakage) - Q0,
    color="blue",
    lw=1.0,
    label=f"Analytical - {"" if leakage else "no "}leakage",
)
plt.xlabel("Time, in hours")
plt.ylabel("Relative Discharge,\nin cubic meters per second")
plt.legend(loc="upper right", fontsize=7)

In [ ]:
dy * 25, dy